In [5]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.svm import SVC
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from nltk.tokenize import WordPunctTokenizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
barsize = '{l_bar}{bar:10}{r_bar}{bar:-10b}'
%matplotlib inline

In [6]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [7]:
# os.listdir('final/')

In [8]:
# data1 = pd.read_csv(r"final/mcintire.csv")
# data2 = pd.read_csv(r"final/buzzfeed.csv")
# data3 = pd.read_csv(r"final/politifact.csv")
# data4 = pd.read_csv(r"final/kaggle.csv")

In [9]:
# df.head()
# dff.head()

In [10]:
# dataset = pd.concat([data1, data2])

In [11]:
# dataset.sample(frac = 1)
# dataset.reset_index(drop =True, inplace = True)

In [12]:
# dataset.label.value_counts()

In [13]:
dataset = pd.read_csv('FINAL_DATASET.csv')
# os.listdir()

In [14]:
real = dataset[dataset['label'] == 'real']
fake = dataset[dataset['label'] == 'fake']
print(len(fake), len(real))

2368 2373


In [15]:
total = 2300
real = real.sample(n = total)
fake = fake.sample(n = total)
print(len(fake), len(real))

2300 2300


In [16]:
dataset = pd.concat([real, fake])

In [17]:
dataset.reset_index(drop =True, inplace = True)
dataset.label.value_counts()

real    2300
fake    2300
Name: label, dtype: int64

In [18]:
import re

In [19]:
dataset.head()
df = dataset.copy()

In [20]:
features = dataset.iloc[:, 3:-2]
dataset = dataset.iloc[:, :3]
features
dataset

,title,text,label
0,Rand Paul: I won the debate last night,"Killing Obama administration rules, dismantlin...",real
1,Here's who's running for president in 2016,"Bio:Secretary of State (2009-2013), Senator fr...",real
2,Why You Shouldn’t (Totally) Freak Out About a ...,If you are sad about the nomination of Donald ...,real
3,A Combative Donald Trump Shows Up at Tampa and...,Some people who know Republican presumptive pr...,real
4,Why I still believe Donald Trump will never be...,As Donald Trump continues his pundit-defying d...,real
...,...,...,...
4595,Fighting Racism Where White Workers Are Hurtin...,Fighting Racism Where White Workers Are Hurtin...,fake
4596,Robert Frank Talks ‘Radical Pragmatism’ and th...,Robert Frank Talks ‘Radical Pragmatism’ and th...,fake
4597,"Fearing Tomorrow’s WikiLeaks Bombshell, Clinto...",NTEB Ads Privacy Policy Fearing Tomorrow’s Wik...,fake
4598,"Space Wars Likely In The Future As US, Russia ...","Videos Space Wars Likely In The Future As US, ...",fake


In [21]:
features

,#symbols,#capital_case,#short,#long,#sentence,polarity,subjectivity,#syllables,#articles,#words,gunning_idx,smog_idx,automated_idx,#deteminants,#verbs,#adjectives,#adverbs,similarity
0,4,6,1,1,2,0.050000,0.183333,137.0,2.0,24,12.93,0.0,16.5,2.0,4.0,3,0,0.357501
1,20,15,1,3,4,0.437273,0.637576,259.0,4.0,51,26.95,0.0,31.2,5.0,10.0,3,1,0.085870
2,144,107,11,40,51,0.024484,0.422063,4140.0,67.0,851,346.79,0.0,430.3,107.0,141.0,84,40,0.191977
3,12,15,1,2,3,0.250000,0.468056,381.0,5.0,66,33.38,0.0,39.0,5.0,6.0,16,1,0.201276
4,424,226,30,96,126,0.156762,0.509255,9731.0,135.0,1955,806.57,0.0,1006.2,240.0,370.0,178,133,0.184433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,114,110,3,23,26,0.049353,0.367972,3566.0,54.0,681,275.43,0.0,342.9,89.0,91.0,72,18,0.320688
4596,269,181,13,66,79,0.134769,0.415635,7349.0,163.0,1641,656.73,0.0,818.2,236.0,219.0,139,116,0.605389
4597,72,157,2,16,18,0.008333,0.565000,2061.0,39.0,410,165.06,0.0,206.3,46.0,57.0,26,17,0.378321
4598,112,168,7,30,37,0.047574,0.415234,3313.0,57.0,619,250.37,0.0,312.3,72.0,85.0,45,29,0.257940


In [22]:
dataset['title'] = dataset['title'].apply(lambda x: re.findall("[A-Za-z0-9]+", str(x)))
dataset['title'] = dataset['title'].apply(lambda x: " ".join(x).lower())
dataset['text'] = dataset['text'].apply(lambda x: re.findall("[A-Za-z0-9]+", str(x)))
dataset['text'] = dataset['text'].apply(lambda x: " ".join(x).lower())

## PREPROCESSING

In [23]:
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [24]:
dataset['title'] = dataset['title'].apply(lambda x : tweet_cleaner(str(x)))
dataset['text'] = dataset['text'].apply(lambda x : tweet_cleaner(str(x)))
# df['meta_data'] = df['meta_data'].apply(tweet_cleaner)

In [25]:
def all_X(row):
    try:
        return row['title']+" "+row["text"]
    except: print("error", row)
        #giving out the error fields.    

In [26]:
dataset["all_X"] = dataset.apply(all_X,axis=1)

In [27]:
dataset

,title,text,label,all_X
0,rand paul i won the debate last night,killing obama administration rules dismantling...,real,rand paul i won the debate last night killing ...
1,here s who s running for president in,bio secretary of state senator from new york f...,real,here s who s running for president in bio secr...
2,why you shouldn t totally freak out about a tr...,if you are sad about the nomination of donald ...,real,why you shouldn t totally freak out about a tr...
3,a combative donald trump shows up at tampa and...,some people who know republican presumptive pr...,real,a combative donald trump shows up at tampa and...
4,why i still believe donald trump will never be...,as donald trump continues his pundit defying d...,real,why i still believe donald trump will never be...
...,...,...,...,...
4595,fighting racism where white workers are hurtin...,fighting racism where white workers are hurtin...,fake,fighting racism where white workers are hurtin...
4596,robert frank talks radical pragmatism and the ...,robert frank talks radical pragmatism and the ...,fake,robert frank talks radical pragmatism and the ...
4597,fearing tomorrow s wikileaks bombshell clinton...,nteb ads privacy policy fearing tomorrow s wik...,fake,fearing tomorrow s wikileaks bombshell clinton...
4598,space wars likely in the future as us russia d...,videos space wars likely in the future as us r...,fake,space wars likely in the future as us russia d...


In [28]:
dataset = dataset[['all_X', 'label']]

In [29]:
print(dataset.columns)

Index(['all_X', 'label'], dtype='object')


In [30]:
#Assign text to X variable and labels to y
X = dataset.all_X
# X = features
y = dataset.label

In [31]:
#Intialize Count Vectorizer
count_vec = CountVectorizer(lowercase = True, stop_words="english", min_df = 2, ngram_range = (1, 1))
#Fit Count Vectorizer
dtm_cv = count_vec.fit_transform(X)
#Convert it to a pandas data frame
df_cv = pd.DataFrame(dtm_cv.toarray(), columns=count_vec.get_feature_names())

In [32]:
new = features.merge(df_cv, right_index = True, left_index = True)
# new = features
new

,#symbols,#capital_case,#short,#long,#sentence,polarity_x,subjectivity_x,#syllables,#articles,#words,...,zu,zucker,zuckerberg,zucman,zuesse,zulu,zurich,zwick,zy,zyuganov
0,4,6,1,1,2,0.050000,0.183333,137.0,2.0,24,...,0,0,0,0,0,0,0,0,0,0
1,20,15,1,3,4,0.437273,0.637576,259.0,4.0,51,...,0,0,0,0,0,0,0,0,0,0
2,144,107,11,40,51,0.024484,0.422063,4140.0,67.0,851,...,0,0,0,0,0,0,0,0,0,0
3,12,15,1,2,3,0.250000,0.468056,381.0,5.0,66,...,0,0,0,0,0,0,0,0,0,0
4,424,226,30,96,126,0.156762,0.509255,9731.0,135.0,1955,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,114,110,3,23,26,0.049353,0.367972,3566.0,54.0,681,...,0,0,0,0,0,0,0,0,0,0
4596,269,181,13,66,79,0.134769,0.415635,7349.0,163.0,1641,...,0,0,0,0,0,0,0,0,0,0
4597,72,157,2,16,18,0.008333,0.565000,2061.0,39.0,410,...,0,0,0,0,0,0,0,0,0,0
4598,112,168,7,30,37,0.047574,0.415234,3313.0,57.0,619,...,0,0,0,0,0,0,0,0,0,0


In [33]:
df_cv.shape
# new.shape
# y.shape

(4600, 32865)

In [34]:
# from sklearn import preprocessing 
# min_max_scaler = preprocessing.StandardScaler() 
# new = min_max_scaler.fit_transform(new) 

In [35]:
# df_cv_scl

In [38]:
x_train, x_test, y_train, y_test = train_test_split(df_cv, y, test_size = 0.3, random_state = 0)

In [39]:
print(x_train.shape,
     x_test.shape)

(3220, 32865) (1380, 32865)


### Naive Bayes

In [40]:
#Initialize model
import time
tic = time.time()
model = MultinomialNB()
#Fit model with df_cv and y
model.fit(x_train, y_train)
#score the model
y_pred = model.predict(x_test)
toc = time.time()
print(toc - tic)
# accuracy = cross_val_score(estimator = model, X = x_train, y = y_train, cv=10)
print("Model accuracy : {:0.2f}%".format(accuracy_score(y_pred,y_test)*100))
# print("cross validation : {:0.2f}%".format(accuracy.mean()*100))
print(classification_report(y_test, y_pred))

2.9474165439605713
Model accuracy : 88.19%
              precision    recall  f1-score   support

        fake       0.90      0.85      0.88       672
        real       0.87      0.91      0.89       708

    accuracy                           0.88      1380
   macro avg       0.88      0.88      0.88      1380
weighted avg       0.88      0.88      0.88      1380



### SVM

In [41]:
tic = time.time()
svm = SVC(C=100, gamma = 0.0001)
svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)
toc = time.time()
print(toc - tic)
# tic = time.time()
# accuracy = cross_val_score(estimator = svm, X = x_train, y = y_train, cv=10)
# toc = time.time()
# print(toc - tic)
print("Model accuracy : {:0.2f}%".format(accuracy_score(y_pred,y_test)*100))
# print("cross validation : {:0.2f}%".format(accuracy.mean()*100))
print(classification_report(y_test, y_pred))

313.228600025177
Model accuracy : 91.38%
              precision    recall  f1-score   support

        fake       0.90      0.93      0.91       672
        real       0.93      0.90      0.91       708

    accuracy                           0.91      1380
   macro avg       0.91      0.91      0.91      1380
weighted avg       0.91      0.91      0.91      1380



In [29]:
# # defining parameter range 
# param_grid = {'C': [0.1, 1, 10, 100, 1000],  
#                'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
#               'kernel': ['rbf']}  
# grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
# grid.fit(x_train, y_train)
# # print best parameter after tuning 
# print(grid.best_params_) 
# print(grid.best_estimator_)

### Ada Boost

In [36]:
import time
tic = time.time()
abc = AdaBoostClassifier(n_estimators = 50,learning_rate = 1)
abc.fit(x_train, y_train)
y_pred = abc.predict(x_test)
toc = time.time()
print(toc - tic)
# tic = time.time()
# acuracy = cross_val_score(estimator = abc, X = x_train, y = y_train, cv = 10)
# toc = time.time()
# print(toc - tic)
y_pred = abc.predict(x_test)
print("Model accuracy : {:0.2f}%".format(accuracy_score(y_pred,y_test)*100))
# print("cross validation : {:0.2f}%".format(acuracy.mean()*100))
print(classification_report(y_test, y_pred))

50.223576068878174
Model accuracy : 85.56%
              precision    recall  f1-score   support

        fake       0.80      0.91      0.85       407
        real       0.92      0.81      0.86       493

    accuracy                           0.86       900
   macro avg       0.86      0.86      0.86       900
weighted avg       0.86      0.86      0.86       900



### Boosting

In [31]:
# gbc = GradientBoostingClassifier(max_depth = 2)
# gbc.fit(x_train, y_train)
# y_pred = gbc.predict(x_test)
# accuracy = cross_val_score(estimator = gbc, X = x_train, y = y_train, cv=10)
# print("Model accuracy : {:0.2f}%".format(accuracy_score(y_pred,y_test)*100))
# print("cross validation : {:0.2f}%".format(accuracy.mean()*100))

In [32]:
# print(classification_report(y_test, y_pred))

### Decision Tree

In [33]:
# dTree = DecisionTreeClassifier()
# dTree.fit(x_train,y_train)
# y_pred = dTree.predict(x_test)
# accuracy = cross_val_score(estimator = dTree, X = x_train, y = y_train, cv = 10)
# print("Model accuracy : {:0.2f}%".format(accuracy_score(y_pred,y_test)*100))
# print("cross validation : {:0.2f}%".format(accuracy.mean()*100))

In [34]:
# path = dTree.cost_complexity_pruning_path(x_train, y_train)
# ccp_alphas = path.ccp_alphas

# trees = []
# for ccp_alpha in tqdm(ccp_alphas, bar_format = barsize):
#     tree  = DecisionTreeClassifier(random_state = 0, ccp_alpha = ccp_alpha)
#     tree.fit(x_train, y_train)
#     trees.append(tree)
    
# train_score = [tree.score(x_train, y_train) for tree in tqdm(trees, bar_format = barsize)]
# test_score = [tree.score(x_test, y_test) for tree in tqdm(trees, bar_format = barsize)]
# # cross_val_scores = [cross_val_score(estimator = tree, X = x_train, y = y_train, cv = 4).mean() for tree in tqdm(trees, bar_format = barsize)]

# fig, ax = plt.subplots()
# ax.set_xlabel('alpha')
# ax.set_ylabel('accuracy')
# ax.set_title('accuracy vs alpha for training and testing sets')
# ax.plot(ccp_alphas, train_score, marker = 'o', label = 'train')
# ax.plot(ccp_alphas, test_score, marker = 'x', label = 'test')
# # ax.plot(ccp_alphas, cross_val_scores, marker = '*', label = 'cross_val')
# plt.legend(['train', 'test', 'cross'])
# plt.show()

In [35]:
# dTree = DecisionTreeClassifier(ccp_alpha = 0.03)
# dTree.fit(x_train,y_train)
# y_pred = dTree.predict(x_test)
# # accuracy = cross_val_score(estimator = dTree, X = x_train, y = y_train, cv = 10)
# print("Model accuracy : {:0.2f}%".format(accuracy_score(y_pred,y_test)*100))
# print("cross validation : {:0.2f}%".format(accuracy.mean()*100))

In [36]:
# print(classification_report(y_test, y_pred))

### KNN

In [37]:
# #neighbors randomly taken
# knn = KNeighborsClassifier(n_neighbors = 7) 
# knn.fit(x_train, y_train) 
# y_pred = knn.predict(x_test)
# accuracy = cross_val_score(estimator = knn, X = x_train, y = y_train, cv = 10)
# print("Model accuracy : {:0.2f}%".format(accuracy_score(y_pred,y_test)*100))
# print("cross validation : {:0.2f}%".format(accuracy.mean()*100))

In [38]:
# error_rate = []  
# # Will take some time 
# for i in tqdm(range(1, 40), bar_format = barsize): 
      
#     knn = KNeighborsClassifier(n_neighbors = i) 
#     knn.fit(x_train, y_train) 
#     pred_i = knn.predict(x_test) 
#     error_rate.append(np.mean(pred_i != y_test)) 

# plt.figure(figsize =(10, 6)) 
# plt.plot(range(1, 40), error_rate, color ='blue', 
#                 linestyle ='dashed', marker ='o', 
#          markerfacecolor ='red', markersize = 10) 
  
# plt.title('Error Rate vs. K Value') 
# plt.xlabel('K') 
# plt.ylabel('Error Rate') 

In [39]:
#error rate for k=1 is very high
#error rate is decreaing after k = 9

In [40]:
# knn = KNeighborsClassifier(n_neighbors = 36) 
# knn.fit(x_train, y_train) 
# y_pred = knn.predict(x_test)
# accuracy = cross_val_score(estimator = knn, X = x_train, y = y_train, cv=10)
# print("Model accuracy : {:0.2f}%".format(accuracy_score(y_pred,y_test)*100))
# print("cross validation : {:0.2f}%".format(acuracy.mean()*100))

In [41]:
# print(classification_report(y_test, y_pred))